## Self-Managed LLM Memory

### Table of Contents

- [Introduction](#introduction)
- [Getting Started](#getting-started)
- [Memory Implementations](#memory-implementations)
    - [Basic Memory](#implementation-1-simple-memory-tool)
    - [Compactify Memory](#implementation-2-compactify-memory)
    - [File-Based Memory](#implementation-3-file-based-memory)
- [Basic Evaluations](#basic-evaluations)
- [Future Work](#future-work)

### Introduction

Managing memory effectively is a critical part of building agents and agentic workflows that handle long-horizon tasks. In this cookbook we demonstrate a few different strategies for "self-managed" (LLM-managed) memory. Use this notebook as a starting point for your own memory implementations. We do not expect that memory tools are one-size-fits-all, and further believe that different domains/tasks necessarily lend themselves to more or less rigid memory scaffolding. The Claude 4 model family has proven to be particularly strong at utilizing [memory tooling](https://www.anthropic.com/news/claude-4#:~:text=more%20on%20methodology.-,Model%20improvements,-In%20addition%20to), and we're excited to see how teams extend the ideas below.


#### Why do we need to manage memory?

LLMs have finite context windows (200k tokens for Claude 4 Sonnet & Opus). This means that for any request, if the sum of prompt tokens and output tokens exceeds the model’s context window, the system will return a validation error. As many teams building with LLMs quickly learn, there is additional complexity in identifying and working within the *effective* [context window](https://docs.anthropic.com/en/docs/build-with-claude/context-windows) of an LLM. See our tips for [long context prompting](https://docs.anthropic.com/en/docs/build-with-claude/prompt-engineering/long-context-tips) to learn more about effective context windows and best practices.

In addition to the above, memory is important for the following reasons:
-  **Long context windows are computationally expensive:** Attention mechanisms scale quadratically—doubling context length quadruples compute cost. Most tasks only need a small fraction of available context, making it wasteful to process millions of irrelevant tokens. This is why humans don't memorize entire textbooks; we take notes and build mental models instead.
- **More efficient processing:** When LLMs write and maintain their own notes—saving successful strategies, key insights, and relevant context—they're effectively updating their capabilities in real-time without retraining. Models that excel at these operations can maintain coherent behavior over extremely long time horizons while using only a fraction of the computational resources required for full context windows.

Successfully building LLM-based systems is an exercise in discarding the unnecessary tokens and efficiently storing + retrieving the relevant tokens for the task at-hand.

### Getting Started

In [ ]:
# install deps
%pip install -q -U anthropic python-dotenv nest_asyncio PyPDF2

In [36]:
# env setup
from anthropic import Anthropic
from dotenv import load_dotenv
import os

# api key must be in .env file in project
load_dotenv()
if os.getenv("ANTHROPIC_API_KEY") is None:
    raise ValueError("ANTHROPIC_API_KEY not found in .env file")

client = Anthropic()

<b>Clone the agents quickstart implementation</b>

We are going to use some of the core work from the agents quickstart implementation which can be found [here](https://github.com/anthropics/anthropic-quickstarts/tree/main/agents).

In [37]:
import sys 
import os

# Check if the repo already exists
if not os.path.exists('/tmp/anthropic-quickstarts'):
    # Clone the agents quickstart implementation
    !git clone https://github.com/anthropics/anthropic-quickstarts.git /tmp/anthropic-quickstarts
else:
    print("Repository already exists at /tmp/anthropic-quickstarts")

# IMPORTANT: Insert at the beginning of sys.path to override any existing 'agents' modules
if '/tmp/anthropic-quickstarts' not in sys.path:
    sys.path.insert(0, '/tmp/anthropic-quickstarts')

# Clear any cached imports of 'agents' module
if 'agents' in sys.modules:
    del sys.modules['agents']
if 'agents.agent' in sys.modules:
    del sys.modules['agents.agent']

Repository already exists at /tmp/anthropic-quickstarts


<b>Confirm the agents repo import works as expected.</b>

In [38]:
import nest_asyncio
nest_asyncio.apply()

from agents.agent import Agent

agent = Agent(
    name="MyAgent",
    system="You are an extremely cynical, snarky, and quick-witted customer support agent. Provide short responses to user queries.",
)

response = agent.run("I'm having issues with my laptop. Can you help me?")
print(response.content[0].text)


*eye roll* Another laptop crisis. What's it doing? Singing off-key? Refusing to work unless you feed it cookies? Details, please.


### Implementation 1: Simple Memory Tool

*This implementation is a reflection of our agents quickstarts repo [here](https://github.com/anthropics/anthropic-quickstarts/tree/main/agents/tools). For more information on tool use, see the Anthropic API tools [docs](https://docs.anthropic.com/en/docs/build-with-claude/tool-use/overview).*

The `SimpleMemory()` tool gives the model a scratchpad to manage memory. This is maintained as a single string that can be read or updated.

Here we've defined the `read`, `write`, and `edit` actions. Explicitly defining `read` means the model won't have access to the full contents of memory at every turn. We recommend that if you follow this pattern you introduce a separate, shortened summary or metadata object describing the contents of memory and include that in every request (ideally preventing excessive reads).


<b>When would you use this?</b>

You want to quickly spin up a memory experiment or augment an existing long-context task. Start here if you don't have high conviction around the types of items that need to be stored or if the agent must support many interaction types.

<b><i>General Notes on Tool Use:</i></b> 
- Your tool descriptions should be clear and sufficiently detailed. The best way to guide model behavior around tools is by providing direction as to when / under what conditions tools should be used. 
- If you find that a task requires the agent or workflow manage many (~20+) tools, you may find better performance by introducing a higher level delegation step to route the task to a specialized LLM-step designed around a smaller, logically coupled subset of tools.

In [39]:
# SIMPLE MEMORY TOOL
from agents.tools.base import Tool

class SimpleMemory(Tool):
    """String-based memory tool for storing and modifying persistent text.

    This tool maintains a single in-memory string that can be read,
    replaced, or selectively edited using string replacement. It provides safety
    warnings when overwriting content or when edit operations would affect
    multiple occurrences.
    """

    name = "simple_memory"

    #TODO: Provide additional domain context to guide Claude on the types of items that should be stored
    description = """Tool for managing persistent text memory with read, write and edit operations.
        Read: Retrieves full memory contents as a string
        Write: Replaces entire memory (warns when overwriting existing data)
        Edit: Performs targeted string replacement (warns on multiple matches)"""

    # single tool that exposes 3 distinct abilities
    input_schema = {
        "type": "object",
        "properties": {
            "action": {
                "type": "string",
                "enum": ["read", "write", "edit"],
                "description": "The memory operation to perform: read retrieves current content, write replaces everything, edit performs string replacement",
            },
            "content": {
                "type": "string",
                "description": "Full text content to store when using write action (ignored for read/edit)",
            },
            "old_string": {
                "type": "string",
                "description": "Exact text to find and replace when using edit action (must be unique in memory)",
            },
            "new_string": {
                "type": "string",
                "description": "Replacement text to insert when using edit action",
            },
        },
        "required": ["action"],
    }

    def __init__(self):
        self.full_memory = ""
        self.compressed_memory = "" # not doing anything with this for now
        
    async def execute(self, **kwargs) -> str:
        """Execute the memory tool with provided parameters."""
        action = kwargs.get("action")
        content = kwargs.get("content", "")
        old_string = kwargs.get("old_string", "")
        new_string = kwargs.get("new_string", "")

        if action == "read":
            return self._read_memory()
        elif action == "write":
            print("Writing to memory...")
            return self._write_memory(content)
        elif action == "edit":
            return self._edit_memory(old_string, new_string)
        else:
            return f"Error: Unknown action '{action}'. Valid actions are read, write, edit."

    def _read_memory(self) -> str:
        """Read the current memory contents."""
        return self.full_memory

    def _write_memory(self, content: str) -> str:
        """Replace the entire memory with new content."""
        if self.full_memory:
            previous = self.full_memory
            self.full_memory = content
            return f"Warning: Overwriting existing content. Previous content was:\n{previous}\n\nMemory has been updated successfully."
        self.full_memory = content
        return "Memory updated successfully."

    def _edit_memory(self, old_string: str, new_string: str) -> str:
        """Replace occurrences of old string with new string."""
        if old_string not in self.full_memory:
            return f"Error: '{old_string}' not found in memory."

        old_memory = self.full_memory
        count = old_memory.count(old_string)

        if count > 1:
            return f"Warning: Found {count} occurrences of '{old_string}'. Please confirm which occurrence to replace or use more specific context."

        self.full_memory = self.full_memory.replace(old_string, new_string)
        return f"Edited memory: 1 occurrence replaced."

    def __str__(self) -> str:
        return self.full_memory

### Implementation 2: Compactify Memory 

Maintaining a rolling summary over long interactions is a pattern you might have already built into your application. Generally the implementation looks something like:


1) Set a `token_threshold`. This threshold could be the context window for the model, but generally you would set it lower.
2) Track the current token usage: `system_prompt` + `rolling_summary` (up to step_n) + `message_history[]` (since step_n)
3) When token usage exceeds threshold, summarize using current `rolling_summary` + `message_history[]`. Clear `message_history[]` and reset `rolling_summary`. 

We believe the pattern outlined above works well. The modification we're introducing with this tool is <i>allowing the model</i> to invoke the summarization operation at its own discretion. You might decide to combine these ideas, allowing the model to determine when to summarize but preserve the `token_threshold` + force summarization as a fail safe in tetheh case that Claude doesn't decide to compactify memory in time. 

<b>When would you use this?</b>

Similar to the first implementation, test this tool when you don't have a clear idea of what should be saved. Behaviorally speaking, decision making around when to condense a long running conversation can be more reliably tuned compared to the open-endedness of the first memory tool.

In [40]:
# COMPACTIFY MEMORY TOOL
from agents.utils.history_util import MessageHistory

class CompactifyMemory(Tool):
    """Memory summarization tool.
    
    Summarizes and replaces the existing message history.
    Expects to have access to a message_history object that is shared with the request handler.
    Descriptions should be modified to introduce use-case specific guidance.
    """
    
    name = "compactify_memory"
    description = """The memory compactifier tool will compress the current conversation history (replaces message history entirely). 
    Should be used when there is sufficient information that requires summarization.
    The summary should keep relevant information from any previous summaries.
    """

    input_schema = {
        "type": "object",
        "properties": {},
        "required": []
    }
        
    def __init__(self, client: Anthropic):
        self.client = client
        self.full_memory = ''
        self.compressed_memory = '' # not doing anything with this for now

    def run_compactify (self, message_history: MessageHistory):
        summary = self.client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens = 10000, # modify as needed
            messages=[*message_history.messages, {
                "role": "user",
                "content": """Your task is to summarize the conversation using the previous summary as well as the messages since the last summary. Note that this will replace the previous summary entirely, so be sure to include the most relevant information that should be persisted."""
            }]
        )

        # modify the message history object in place
        message_history.messages = [
            {
                "role": "assistant",
                "content": "Conversation Summary: " +  summary.content[0].text
            }
        ]
        
    async def execute(self, **kwargs) -> str:
        # ATTN: note that we're breaking tool encapsulation here and will be executing the function outside the agent loop (see agents.agent.py)
        # we do this because we don't have an elegant way to share message state between the agent and tool just yet (...stay tuned)
        return "pending_compactify"
        
    def __str__(self):
        return self.full_memory
        

### Implementation 3: "File-Based" Memory

This implementation gives Claude the ability to interact with a 'memory' system represented to the model as a hierarchical file structure. The example below implements a basic directory, where the 'files' are just strings that we've labeled as plaintext files (the '.txt' label has no impact functionally, but can be useful for behavioral consistency).

Hierarchical directory structures are easily readable and well-understood by humans and LLMs alike, so it's fitting to use them as a mechanism to represent persistent state more generally to an LLM. While you can connect to and define access patterns for any external storage system, a quick way to get started is with Anthropic's new <b>[Files API](https://docs.anthropic.com/en/docs/build-with-claude/files)</b>. The Files API enables storage and retrieval of objects for use in future requests.

Ideally you (the developer & domain expert) would construct an initial state for the directory structure that adequately represents your domain context. Having some pre-defined structure provides useful behavioral queues for the model, but you should also introduce more explicit guidance to guard against excessive reads / writes / new file creation / etc.

In [41]:
import json
import re

# HELPER FUNCTION: Parse markdown string for JSON
def parse_markdown_json(markdown_string):
    """
    Parses a JSON string from a Markdown string.

    Args:
        markdown_string (str): The Markdown string containing JSON.

    Returns:
        dict or list or None: A Python object representing the parsed JSON, or None if parsing fails.
    """
    match = re.search(r"```(?:json)?\n(.*?)\n```", markdown_string, re.DOTALL)
    if match:
        json_string = match.group(1).strip()
    else:
        json_string = markdown_string.strip()
    try:
        parsed_json = json.loads(json_string)
        return parsed_json
    except json.JSONDecodeError:
        return None

# HELPER CLASS: Memory Node
class MemoryNode:
    def __init__(self, name, is_directory=False, parent=None, content=None):
        self.name = name
        self.is_directory = is_directory
        self.parent = parent
        self.content = content if not is_directory else None
        self.children = {} if is_directory else None
    
    def add_child(self, name, is_directory=False, content=None):
        """Add a child node to the current node."""
        if not self.is_directory:
            raise ValueError(f"Cannot add child to file '{self.name}'")
        
        if name in self.children:
            raise ValueError(f"Child '{name}' already exists")
        
        child = MemoryNode(name, is_directory, parent=self, content=content)
        self.children[name] = child
        return child
    
    def remove_child(self, name):
        """Remove a child node from the current node."""
        if not self.is_directory:
            raise ValueError(f"Cannot remove child from file '{self.name}'")
            
        if name not in self.children:
            raise ValueError(f"Child '{name}' not found")
        
        del self.children[name]
    
    def find(self, path):
        """Find a node by path (ex: 'folder1/folder2/file.txt')."""
        if not path:
            return self
        
        parts = path.strip('/').split('/', 1)
        child_name = parts[0]
        
        if not self.is_directory or child_name not in self.children:
            return None
            
        child = self.children[child_name]
        
        if len(parts) == 1:
            return child
        else:
            return child.find(parts[1])
    
    def __repr__(self):
        return f"MemoryNode(name='{self.name}', is_directory={self.is_directory})"

# HELPER CLASS: Memory Tree
class MemoryTree:
    def __init__(self):
        self.root = MemoryNode("memory", is_directory=True)

    def add(self, path, content):
        """Add content to a node at the given path (ex: 'folder1/folder2/file.txt')."""
        node = self.root.find(path)
        if node:
            node.content = content
        else:
            raise ValueError(f"Path '{path}' not found")

    def get(self, path):
        """Get content from a node at the given path."""
        node = self.root.find(path)
        if node:
            return node.content
        else:
            raise ValueError(f"Path '{path}' not found")

    def edit(self, path, content):
        node = self.root.find(path)
        if node:
            node.content = content
        else:
            raise ValueError(f"Path '{path}' not found")

    def _build_from_json_recursive(self, json_obj, parent_node):
        """Recursively build the tree from a JSON object."""

        # handle root memory (already initialized)
        if len(json_obj) == 1 and 'memory' in json_obj:
            json_obj = json_obj['memory']

        for name, value in json_obj.items():
            if isinstance(value, dict):
                # Create a directory node
                child_node = parent_node.add_child(name, is_directory=True)
                self._build_from_json_recursive(value, child_node)
            else:
                # Create a file node with content
                parent_node.add_child(name, content=value)

    def build_from_json_string(self, str_json_obj):
        json_obj = parse_markdown_json(str_json_obj)
        self._build_from_json_recursive(json_obj, self.root)

    def print_tree(self, node=None, prefix=''):
        """Print a directory tree structure."""
        if node is None:
            node = self.root
        
        # Build list of children for proper indexing
        children = list(node.children.items()) if node.is_directory else []
        
        for index, (name, child) in enumerate(children):
            is_last = index == len(children) - 1
            
            # Create the appropriate connector
            if prefix == '' and node == self.root:
                # For root level items (direct children of root)
                connector = '└── ' if is_last else '├── '
                self.lines.append(f"{connector}{name}")
                
                # Recurse if this is a directory
                if child.is_directory:
                    extension = '    ' if is_last else '│   '
                    self.print_tree(child, extension)
            else:
                # For non-root level items
                connector = '└── ' if is_last else '├── '
                self.lines.append(f"{prefix}{connector}{name}")
                
                # Recurse if this is a directory
                if child.is_directory:
                    extension = '    ' if is_last else '│   '
                    self.print_tree(child, prefix + extension)

    def get_tree(self):
        """Return the tree as a string."""
        self.lines = []
        
        # Start with the root directory name
        self.lines.append(self.root.name)

        # Print the rest of the tree
        self.print_tree()
        return '\n'.join(self.lines)

    def __str__(self):
        return self.get_tree()

    def __repr__(self):
        return str(self)

In [43]:
import requests
import mimetypes

# HELPER CLASS FOR FILE STORAGE using the new files API!
class StorageManager:
    def __init__(self, api_key):
        if api_key is None:
            raise ValueError("ANTHROPIC_API_KEY not available.")
        self.api_key = api_key
        self.base_url = "https://api.anthropic.com/v1/files"
        self.headers = {
            "x-api-key": self.api_key,
            "anthropic-version": "2023-06-01",
            "anthropic-beta": "files-api-2025-04-14"
        }

    def _execute_request(self, method, endpoint, data=None, files=None):
        """Execute a request to the API."""
        url = f"{self.base_url}/{endpoint}"

        res = requests.request(method, url, headers=self.headers, data=data, files=files)
        if res.status_code == 200:
            return res.json()
        else:
            raise ValueError(f"Request failed: {res.status_code} - {res.text}")

    def list_files(self):
        """List all files. Direct curl request to the API."""
        res = requests.get(
            self.base_url,
            headers=self.headers
        )
        if res.status_code != 200:
            raise ValueError(f"Failed to retrieve files: {res.status_code} - {res.text}")
        res = res.json()
        return res['data']
        
        
    def get_file_metadata(self, file_id):
        """Get a file by ID. Direct curl request to the API."""
        res = requests.get(
            f"{self.base_url}/{file_id}",
            headers=self.headers
        )
        if res.status_code != 200:
            raise ValueError(f"Failed to retrieve file: {res.status_code} - {res.text}")
        res = res.json()
        return res 
        
    def upload_file(self, file_path):
        """Upload a file to the API."""        
        # Determine the file's MIME type
        mime_type, _ = mimetypes.guess_type(file_path)
        if mime_type is None:
            mime_type = "application/octet-stream"  # Fallback to binary if type unknown
        
        with open(file_path, "rb") as file_obj:
            files = {
                "file": (os.path.basename(file_path), file_obj, mime_type)
            }
            
            res = requests.post(
                self.base_url,
                headers=self.headers,
                files=files
            )
            
        if res.status_code == 200:
            return res.json()
        else:
            raise ValueError(f"Failed to upload file: {res.status_code} - {res.text}")
        
# example usage
#file_path = "/Users/user/Downloads/SB1029-ProjectUpdate-FINAL_020317-A11Y.pdf" # REPLACE
storage_manager = StorageManager(os.getenv("ANTHROPIC_API_KEY"))
#uploaded = storage_manager.upload_file(file_path)
#storage_manager.get_file_metadata(uploaded['id'])
storage_manager.list_files()[:2]

[{'type': 'file',
  'id': 'file_011CPaGpXxdBojQLTszA5LGp',
  'size_bytes': 544347,
  'created_at': '2025-05-28T16:51:06.716000Z',
  'filename': 'sample.pdf',
  'mime_type': 'application/pdf',
  'downloadable': False},
 {'type': 'file',
  'id': 'file_011CPYNG2Sf1cWjuCFhKJFV7',
  'size_bytes': 3,
  'created_at': '2025-05-27T16:41:15.335000Z',
  'filename': 'number.txt',
  'mime_type': 'text/plain',
  'downloadable': True}]

#### What does this look like in practice?

Imagine you want to build a company wide chatbot that needs access to information about ongoing projects, teams, customers, etc. You could build a retrieval pipeline that chunks, loads, and refreshes your company documents within a vector database, but tuning this pipeline is a non-trivial task. The neat part about building file-based memory scaffolding for this problem is you can treat files managed by your organization in the exact same manner as files managed by the LLM (just with different read/write permissions).

Imagine the agent has access the following directory at every turn and can read and update these objects at its discretion.

```
claude_memories/
├── user_session_notes/
│   ├── cli_debuggin_session_2025_05_02.txt
│   ├── quarterly_planning_2025_05_01.txt
│   └── data_analysis_2025_05_01.txt
├── general_preferences/
│   ├── code_style.txt
│   └── all_preferences.txt
files/
├── projects/
│   ├── building_agi.txt
│   └── prompt_optimization.txt
├── documents/
│   ├── updated_risk_report.txt
│   ├── company_strategy.txt
│   └── 2024_annual_report.txt
├── teams/
│   ├── engineering.txt
│   └── marketing.txt
├── customers/
│   ├── acme.txt
│   └── widgets.txt
```

A fully featured implementation might enforce the following:
- `claude_memories/` directory (llm-managed) allows <b>read</b> & <b>write</b> operations
- `user_session_notes` is stored + loaded <b>per user</b>
- `files/` directory (org-managed) is <b>read-only</b> and connects to an external storage system
- as directories grow past a certain size you may want to limit traversal up to depth <b><i>n</i></b>, and then allow the model to invoke deeper traversal only as neeeded

*In theory, the Simple Memory tool presented in #1 could be represented as a file system with a single available path.*


In [44]:
# example usage
company_agent_memory = MemoryTree()

# example of the type of object you might get from an LLM (if you wanted to allow the LLM to construct it's own memory structure)
example_str = """
```json
{"self_managed": {"user_session_notes":{"ongoing_projects.txt":"I should remember that the user is working on prompt optimization","preferences.txt":"I should remember that the user prefers to be called Jimbo"},"projects":{"building_agi.txt":"I should remember that the user is working on building AGI"}}, "files": {"projects":"building_agi.txt"}}
```
"""

company_agent_memory.build_from_json_string(example_str)
company_agent_memory

# test out the file utilities below

# print(company_agent_memory)
# print("GET:", company_agent_memory.get('self_managed/user_session_notes/ongoing_projects.txt'))
# company_agent_memory.edit('self_managed/user_session_notes/ongoing_projects.txt', 'The user gave up on prompt optimization')
# print("UPDATED:", company_agent_memory.get('self_managed/user_session_notes/ongoing_projects.txt'))

memory
├── self_managed
│   ├── user_session_notes
│   │   ├── ongoing_projects.txt
│   │   └── preferences.txt
│   └── projects
│       └── building_agi.txt
└── files
    └── projects

In [45]:
# FILE BASED MEMORY TOOL

class FileBasedMemoryTool(Tool):
    """
    Manage memory as a nested file system. This is specifically designed around the new files API.

    This tool provides a simple interace for interacting with this memory system.
    We have only defined three actions: GET, EDIT, and BUILD. In practice, you likely would opt for a more opinionated file structure 
    and more fine-grained control over access to the memory. We will rely on the default message truncation mechanism of the request handler.
    """

    name = 'hierarchical_memory'
    description = 'Interact with file system for storing memories, retrieving memories, and rebuilding the memory state.'
    input_schema = {
        'type': 'object',
        'properties': {
            'action': {
                'type': 'string',
                'enum': ['get', 'edit', 'build']
            },
            'paths': {
                'type': 'array',
                'items': {
                    'type': 'string',
                    'description': 'Path to the memory item'
                },
                'description': 'List of paths for the associated action. Available with GET and EDIT actions. (GET can have multiple paths, EDIT should have one path)'
            },
            'content': {
                'type': 'string',
                'description': 'Content that will be written to the specified path. Only available with the EDIT action.'
            },
            'new_memory_object': {
                'type': 'object',
                'description': 'Full memory output object to rebuild the memory scaffold. Only available with the BUILD action. This should be a JSON object representing the desired tree structure for memories. The values should be None (as a placeholder for future content).'
            }
        },
        'required': ['action']
    }
    
    def __init__(self, storage_manager: StorageManager):
        self.full_memory = MemoryTree()
        self.compressed_memory = self.full_memory # including the compressed memory for standardizing the interface
        self.storage_manager = storage_manager

    async def execute(self, **kwargs) -> str:
        action = kwargs.get('action')
        paths = kwargs.get('paths')
        content = kwargs.get('content')
        new_memory_object = kwargs.get('new_memory_object')

        if action == 'get':
            # we need to build the file messages from the file metadata (https://docs.anthropic.com/en/docs/docs/build-with-claude/files)
            message_refs = [{"type": "document", "source": { "type": "file", "file_id": self.full_memory.get(path)}} for path in paths]
            return message_refs

        elif action == 'edit':
            path = paths[0]

            #create txt file in tmp dir with content
            with open(f'/tmp/{path}.txt', 'w') as f:
                f.write(content)

            # upload the file to the API
            uploaded = self.storage_manager.upload_file(f'/tmp/{path}.txt')

            # add the file to the memory tree (using the id)
            self.full_memory.edit(path, uploaded['id'])
            return 'Updated'
        
        elif action == 'build':
            self.full_memory.build_from_json_string(new_memory_object)
            return 'Updated'
        
        else:
            raise ValueError(f"Invalid action: {action}")
        
    def __str__(self):
        return str(self.memory)

#### General Memory Management Advice:
- Maintain a summary or compressed representation of memory preloaded in the context, even if your tools require actions from the model to load the full information.

- Encourage the model to reason about what to remember and how to update its memory content given the domain or task at hand.

- Encourage the model to keep the content of its memory up-to-date and coherent. Discourage excessive file creation.

### An Interactive Demo

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import datetime
import textwrap
from typing import List
from anthropic import Anthropic

memory_tools = [
    SimpleMemory(),
    CompactifyMemory(client),
    FileBasedMemoryTool()
]

def process_memory_function(agent, tool):
    """Because some memory tools work with the agents message history object"""
    mem_tool_names = [tool.name for tool in memory_tools]
    for tool in agent.tools:
        if tool.name in mem_tool_names:
            # ATTN: bit of a hack, but we need to inject some additional functionality
            if tool.name == 'compactify_memory':
                tool.run_compactify(self.agent.message_history)
        

class ChatInterface:
    def __init__(self, agent: Agent, max_line_length=80):
        self.max_line_length = max_line_length
        self.agent = agent
        self.messages = [] # managing the window's messages separately from the Agent's messages
        self.memory = ''

        # Chat history container
        self.chat_output = widgets.Output(layout=widgets.Layout(
            height='400px', 
            overflow='auto',
            border='1px solid #ccc',
            padding='10px',
            display='flex',
            flex_flow='wrap-reverse'
        ))
        
        # Text input for new messages
        self.text_input = widgets.Text(
            placeholder='Type your message here...',
            layout=widgets.Layout(width='100%')
        )
        
        # Send button
        self.send_button = widgets.Button(
            description='Send',
            button_style='primary'
        )
        
        # Memory settings display
        self.memory_display = widgets.Output(layout=widgets.Layout(
            width='100%', 
            height='400px',
            border='1px solid #ccc',
            padding ='10px',

        ))  
        
        # Input container (text input + send button)
        input_box = widgets.HBox([
            self.text_input,
            self.send_button
        ], layout=widgets.Layout(width='100%'))
        
        # Left panel (chat)
        left_panel = widgets.VBox([
            widgets.Label('Chat'),
            self.chat_output,
            input_box
        ], layout=widgets.Layout(
            width='50%',
            padding='10px',
        ))
        
        # Right panel (memory settings)
        right_panel = widgets.VBox([
            widgets.Label('Memory'),
            self.memory_display
        ], layout=widgets.Layout(
            width='50%',
            padding='10px'
        ))
        
        # Main layout
        self.interface = widgets.HBox([
            left_panel,
            right_panel
        ], layout=widgets.Layout(
            width='100%',
            display='flex'
        ))
        
        # Event handlers
        self.send_button.on_click(self.on_send)
        self.text_input.on_submit(self.on_send)
        
        # Message history
        self.messages = []
    
    def on_send(self, _):
        """Handle sending a message"""
        message = self.text_input.value.strip()
        if message:
            self.add_message("user", message)
            self.text_input.value = ""

            # call the agent with the message
            response = self.agent.run(message)
            self.add_message("assistant", response.content[0].text)

            ## PROCESS

            self.update_memory_display()
    
    def wrap_text(self, text):
        """Wrap text to fit within max_line_length"""
        # Use textwrap to wrap long lines
        wrapped_lines = []
        for line in text.split('\n'):
            if len(line) > self.max_line_length:
                # Wrap this line
                wrapped = textwrap.fill(line, width=self.max_line_length)
                wrapped_lines.append(wrapped)
            else:
                wrapped_lines.append(line)
        return '\n'.join(wrapped_lines)
    
    def add_message(self, role, message):
        """Add a message to the chat history with text wrapping"""
        timestamp = datetime.datetime.now().strftime("%H:%M:%S")
        # Wrap the message text
        wrapped_message = self.wrap_text(message)
        
        self.messages.append({
            "role": role,
            "content": message,  # Store original message
            "wrapped_message": wrapped_message,  # Store wrapped version
            "timestamp": timestamp
        })
        
        with self.chat_output:
            clear_output()
            # Display all messages with HTML formatting
            for msg in self.messages:
                if msg['role'] == 'user':
                    color = '#0066cc'
                else:
                    color = '#000000'
                    
                display(HTML(
                    f"<div style='margin-bottom: 10px; color: {color};'>"
                    f"<strong>{msg['role']} [{msg['timestamp']}]:</strong> "
                    f"{msg['wrapped_message']}"
                    f"</div>"
                ))
            
    def update_memory_display(self):
        """Update the memory display with current memory content"""
        with self.memory_display:
            clear_output()
            display(HTML(f"<pre style='margin: 10px; padding: 0; white-space: pre-wrap;'>{self.memory}</pre>"))
    
    def display(self):
        """Display the interface"""
        return self.interface

#### Run The Demo

In [ ]:
memory_tool = FileBasedMemoryTool() # or SimpleMemory() or CompactifyMemory(client) or FileBasedMemoryTool(storage_manager)
model_config = {
    "model": "claude-sonnet-4-20250514",
}
agent = Agent(
    name="Assistant",
    system="You are a helpful assistant designed to work with a user.", # additional memory instructions can be added here
    tools=[memory_tool],
    config=model_config,
)

chat = ChatInterface(
    agent=agent,
)

chat.display()